In [3]:

# #  INFORME DEL MERCADO DE HIDROCARBUROS
# ## Gerencia de Mercados y Proyectos - Sector Energía
# ### Análisis en Tiempo Real: Crudo, GN, GLP y Derivados
#
# **Fecha de actualización:** Datos en tiempo real
# **Período de análisis:** Último mes + tendencias históricas

# %%
# Instalación de librerías necesarias
!pip install yfinance pandas-datareader plotly fredapi seaborn --quiet

# %%
# Importación de librerías
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
import warnings
warnings.filterwarnings('ignore')

print(" Librerías cargadas correctamente")


# ## 🔄 EXTRACCIÓN DE DATOS EN TIEMPO REAL

# %%
# Configuración de períodos de tiempo
end_date = datetime.now()
start_date = end_date - timedelta(days=180)

print("📡 Conectando a fuentes de datos en tiempo real...")

# %%
# Extracción de datos de commodities energéticos
def obtener_datos_mercado():
    """
    Función para extraer datos en tiempo real de principales hidrocarburos
    """

    # Símbolos de Yahoo Finance para commodities energéticos
    simbolos = {
        'Crudo_WTI': 'CL=F',        # WTI Crude
        'Crudo_Brent': 'BZ=F',      # Brent Crude
        'Gas_Natural': 'NG=F',      # Natural Gas
        'Gasolina': 'RB=F',         # RBOB Gasoline
        'Fuel_Oil': 'HO=F',         # Heating Oil
        'Oro': 'GC=F'              # Gold como referencia (GLP no está disponible)
    }

    datos = {}
    for nombre, simbolo in simbolos.items():
        try:
            temp_data = yf.download(simbolo, start=start_date, end=end_date, progress=False)
            if not temp_data.empty:
                # Verificar qué columnas están disponibles
                available_columns = temp_data.columns.tolist()
                print(f"✅ {nombre}: {len(temp_data)} registros - Columnas: {available_columns}")

                # Usar 'Close' si 'Adj Close' no está disponible
                if 'Adj Close' in temp_data.columns:
                    datos[nombre] = temp_data
                else:
                    # Crear una copia y usar 'Close' como 'Adj Close'
                    temp_data_adj = temp_data.copy()
                    temp_data_adj['Adj Close'] = temp_data['Close']
                    datos[nombre] = temp_data_adj
            else:
                print(f"⚠️ {nombre}: Sin datos disponibles")
        except Exception as e:
            print(f"❌ Error en {nombre}: {str(e)}")

    return datos

# Ejecutar extracción de datos
datos_mercado = obtener_datos_mercado()


# ## 📈 ANÁLISIS DE PRECIOS Y VOLATILIDAD

# %%
# Análisis de precios de cierre
def analizar_precios(datos):
    """
    Análisis completo de precios y volatilidad
    """
    precios_cierre = pd.DataFrame()

    for nombre, df in datos.items():
        if not df.empty and 'Adj Close' in df.columns:
            precios_cierre[nombre] = df['Adj Close']
        elif not df.empty and 'Close' in df.columns:
            precios_cierre[nombre] = df['Close']

    # Verificar que tenemos datos
    if precios_cierre.empty:
        print("❌ No se pudieron obtener datos de precios")
        return None

    # Estadísticas descriptivas
    print("📊 ESTADÍSTICAS DESCRIPTIVAS - ÚLTIMOS 6 MESES")
    print("=" * 60)
    stats = precios_cierre.describe()
    print(stats.round(2))

    return precios_cierre

precios_cierre = analizar_precios(datos_mercado)

# %%
# Cálculo de volatilidad (desviación estándar móvil 20 días)
def calcular_volatilidad(precios):
    """
    Cálculo de volatilidad histórica
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para calcular volatilidad")
        return None

    volatilidad = precios.pct_change().rolling(window=20).std() * np.sqrt(252) * 100

    fig = go.Figure()
    for columna in volatilidad.columns:
        if not volatilidad[columna].isna().all():  # Solo columnas con datos
            fig.add_trace(go.Scatter(
                x=volatilidad.index,
                y=volatilidad[columna],
                name=columna,
                mode='lines'
            ))

    fig.update_layout(
        title='📊 VOLATILIDAD ANUALIZADA DE HIDROCARBUROS (20 días)',
        xaxis_title='Fecha',
        yaxis_title='Volatilidad Anualizada (%)',
        template='plotly_white',
        height=500
    )

    fig.show()

    return volatilidad

volatilidad = calcular_volatilidad(precios_cierre)


# ## 📊 VISUALIZACIÓN DE PRECIOS COMPARATIVOS

# %%
# Gráfico interactivo de precios normalizados
def grafico_precios_normalizados(precios):
    """
    Gráfico de precios normalizados (base 100) para comparación
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para generar gráfico")
        return

    precios_normalizados = (precios / precios.iloc[0]) * 100

    fig = go.Figure()

    for columna in precios_normalizados.columns:
        fig.add_trace(go.Scatter(
            x=precios_normalizados.index,
            y=precios_normalizados[columna],
            name=columna,
            mode='lines',
            hovertemplate='<b>%{x}</b><br>Precio: %{y:.1f} (base 100)<extra></extra>'
        ))

    fig.update_layout(
        title='🚀 EVOLUCIÓN COMPARATIVA DE PRECIOS (Base 100)',
        xaxis_title='Fecha',
        yaxis_title='Precio Normalizado (Base 100)',
        template='plotly_dark',
        height=600,
        showlegend=True
    )

    fig.show()

grafico_precios_normalizados(precios_cierre)

# %%
# Heatmap de correlaciones
def heatmap_correlaciones(precios):
    """
    Mapa de calor de correlaciones entre commodities
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para calcular correlaciones")
        return None

    correlaciones = precios.corr()

    fig = go.Figure(data=go.Heatmap(
        z=correlaciones.values,
        x=correlaciones.columns,
        y=correlaciones.index,
        colorscale='RdBu_r',
        zmin=-1, zmax=1,
        hoverongaps=False,
        text=correlaciones.round(3).values,
        texttemplate="%{text}",
        textfont={"size": 10}
    ))

    fig.update_layout(
        title='🔥 MAPA DE CORRELACIONES ENTRE HIDROCARBUROS',
        xaxis_title='Commodities',
        yaxis_title='Commodities',
        template='plotly_white',
        height=500
    )

    fig.show()

    return correlaciones

correlaciones = heatmap_correlaciones(precios_cierre)

# ## 📉 ANÁLISIS TÉCNICO - TENDENCIAS Y MOVIMIENTOS

# %%
# Análisis de tendencias con medias móviles
def analisis_tendencias(datos):
    """
    Análisis de tendencias usando medias móviles
    """
    for nombre, df in datos.items():
        if not df.empty:
            # Determinar la columna de precio a usar
            precio_col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'

            fig = go.Figure()

            # Precio de cierre
            fig.add_trace(go.Scatter(
                x=df.index, y=df[precio_col],
                name='Precio Cierre',
                line=dict(color='blue', width=2)
            ))

            # Media móvil 20 días
            mm_20 = df[precio_col].rolling(window=20).mean()
            fig.add_trace(go.Scatter(
                x=df.index, y=mm_20,
                name='MM 20 días',
                line=dict(color='orange', width=1.5)
            ))

            # Media móvil 50 días
            mm_50 = df[precio_col].rolling(window=50).mean()
            fig.add_trace(go.Scatter(
                x=df.index, y=mm_50,
                name='MM 50 días',
                line=dict(color='red', width=1.5)
            ))

            fig.update_layout(
                title=f'📈 {nombre} - ANÁLISIS DE TENDENCIAS',
                xaxis_title='Fecha',
                yaxis_title='Precio (USD)',
                template='plotly_white',
                height=400
            )

            fig.show()

analisis_tendencias(datos_mercado)


# ## 💹 ANÁLISIS DE RENTABILIDAD Y PERFORMANCE

# %%
# Análisis de retornos y performance
def analisis_performance(precios):
    """
    Análisis de retornos y métricas de performance
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para análisis de performance")
        return None, None

    retornos = precios.pct_change().dropna()

    # Métricas de performance
    performance = pd.DataFrame({
        'Retorno_Total (%)': (precios.iloc[-1] / precios.iloc[0] - 1) * 100,
        'Volatilidad_Anual (%)': retornos.std() * np.sqrt(252) * 100,
        'Sharpe_Ratio': (retornos.mean() / retornos.std()) * np.sqrt(252),
        'Max_Drawdown (%)': (precios / precios.expanding().max() - 1).min() * 100
    })

    print("🎯 MÉTRICAS DE PERFORMANCE - ÚLTIMOS 6 MESES")
    print("=" * 70)
    print(performance.round(3))

    # Gráfico de retornos acumulados
    retornos_acumulados = (1 + retornos).cumprod()

    fig = go.Figure()
    for columna in retornos_acumulados.columns:
        fig.add_trace(go.Scatter(
            x=retornos_acumulados.index,
            y=retornos_acumulados[columna] * 100,
            name=columna,
            mode='lines'
        ))

    fig.update_layout(
        title='📊 RETORNOS ACUMULADOS (%)',
        xaxis_title='Fecha',
        yaxis_title='Retorno Acumulado (%)',
        template='plotly_white',
        height=500
    )

    fig.show()

    return performance, retornos

performance, retornos = analisis_performance(precios_cierre)


# ## 📋 INFORME EJECUTIVO - RESUMEN GERENCIAL

# %%
# Generación de informe ejecutivo
def generar_informe_ejecutivo(precios, performance, volatilidad):
    """
    Generación de informe ejecutivo resumido
    """
    if precios is None or precios.empty:
        print("❌ No hay datos suficientes para generar informe ejecutivo")
        return

    print("🎯 INFORME EJECUTIVO - MERCADO DE HIDROCARBUROS")
    print("=" * 70)
    print(f"📅 Período analizado: {start_date.strftime('%Y-%m-%d')} a {end_date.strftime('%Y-%m-%d')}")
    print(f"⏰ Última actualización: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Commodities analizados: {', '.join(precios.columns)}")
    print("\n")

    # Precios actuales vs iniciales
    print("💰 EVOLUCIÓN DE PRECIOS:")
    print("-" * 40)
    for commodity in precios.columns:
        precio_inicial = precios[commodity].iloc[0]
        precio_actual = precios[commodity].iloc[-1]
        variacion = ((precio_actual - precio_inicial) / precio_inicial) * 100

        tendencia = "🟢 ALZA" if variacion > 0 else "🔴 BAJA"
        print(f"{commodity:<15}: {precio_actual:8.2f} USD ({variacion:+.2f}%) {tendencia}")

    print("\n")

    if performance is not None:
        # Commodity con mejor y peor performance
        mejor = performance['Retorno_Total (%)'].idxmax()
        peor = performance['Retorno_Total (%)'].idxmin()

        print("🏆 PERFORMANCE DESTACADA:")
        print("-" * 40)
        print(f"Mejor desempeño: {mejor} ({performance.loc[mejor, 'Retorno_Total (%)']:.2f}%)")
        print(f"Peor desempeño: {peor} ({performance.loc[peor, 'Retorno_Total (%)']:.2f}%)")

        print("\n")

    if volatilidad is not None and not volatilidad.empty:
        # Volatilidad actual
        print("🌪️  VOLATILIDAD ACTUAL:")
        print("-" * 40)
        volatilidad_actual = volatilidad.iloc[-1]
        for commodity in precios.columns:
            if commodity in volatilidad_actual.index:
                vol = volatilidad_actual[commodity]
                nivel_vol = "Alta" if vol > 40 else "Media" if vol > 20 else "Baja"
                print(f"{commodity:<15}: {vol:6.2f}% ({nivel_vol})")

        print("\n")

    # Recomendaciones basadas en análisis
    print("💡 RECOMENDACIONES ESTRATÉGICAS:")
    print("-" * 40)

    # Análisis de tendencias para recomendaciones
    for commodity in precios.columns:
        precio_actual = precios[commodity].iloc[-1]
        mm_20 = precios[commodity].rolling(20).mean().iloc[-1]
        mm_50 = precios[commodity].rolling(50).mean().iloc[-1]

        if precio_actual > mm_20 and precio_actual > mm_50:
            print(f"• {commodity}: TENDENCIA ALCISTA 🟢")
        elif precio_actual < mm_20 and precio_actual < mm_50:
            print(f"• {commodity}: TENDENCIA BAJISTA 🔴")
        else:
            print(f"• {commodity}: TENDENCIA LATERAL 🟡")

generar_informe_ejecutivo(precios_cierre, performance, volatilidad)


# ## 📊 DASHBOARD INTERACTIVO RESUMEN

# %%
# Creación de dashboard resumen
def crear_dashboard_resumen(precios, performance, volatilidad):
    """
    Dashboard interactivo con métricas clave
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para crear dashboard")
        return

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '📈 Precios Actuales vs Iniciales',
            '📊 Retorno Total (%)',
            '🌪️ Volatilidad Anualizada (%)',
            '⚡ Sharpe Ratio'
        ),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )

    # Gráfico 1: Variación porcentual
    precios_actuales = precios.iloc[-1]
    precios_iniciales = precios.iloc[0]
    variaciones = ((precios_actuales - precios_iniciales) / precios_iniciales) * 100

    colors = ['green' if x > 0 else 'red' for x in variaciones]
    fig.add_trace(
        go.Bar(x=variaciones.index, y=variaciones.values,
               marker_color=colors, name='Variación %'),
        row=1, col=1
    )

    # Gráfico 2: Retorno total
    if performance is not None:
        fig.add_trace(
            go.Bar(x=performance.index, y=performance['Retorno_Total (%)'],
                   marker_color='blue', name='Retorno Total'),
            row=1, col=2
        )

    # Gráfico 3: Volatilidad
    if volatilidad is not None and not volatilidad.empty:
        volatilidad_actual = volatilidad.iloc[-1]
        fig.add_trace(
            go.Bar(x=volatilidad_actual.index, y=volatilidad_actual.values,
                   marker_color='orange', name='Volatilidad'),
            row=2, col=1
        )

    # Gráfico 4: Sharpe Ratio
    if performance is not None:
        fig.add_trace(
            go.Bar(x=performance.index, y=performance['Sharpe_Ratio'],
                   marker_color='purple', name='Sharpe Ratio'),
            row=2, col=2
        )

    fig.update_layout(
        title_text='📊 DASHBOARD EJECUTIVO - MERCADO HIDROCARBUROS',
        height=600,
        showlegend=False,
        template='plotly_white'
    )

    fig.show()

crear_dashboard_resumen(precios_cierre, performance, volatilidad)


# ## 🔮 PERSPECTIVAS Y OUTLOOK

# %%
# Análisis de perspectivas
def analisis_perspectivas(precios, performance):
    """
    Análisis de perspectivas basado en tendencias recientes
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para análisis de perspectivas")
        return

    print("🔮 PERSPECTIVAS Y OUTLOOK DE MERCADO")
    print("=" * 50)
    print("\n")

    # Análisis de tendencias recientes (últimos 30 días)
    precios_recientes = precios.tail(30)
    tendencias_recientes = {}

    for commodity in precios_recientes.columns:
        x = np.arange(len(precios_recientes))
        y = precios_recientes[commodity].values
        z = np.polyfit(x, y, 1)
        tendencias_recientes[commodity] = z[0]  # Pendiente

    print("📈 TENDENCIAS RECIENTES (Últimos 30 días):")
    print("-" * 45)

    for commodity, pendiente in tendencias_recientes.items():
        if pendiente > 0:
            outlook = "🟢 POSITIVO"
        elif pendiente < -0.5:
            outlook = "🔴 CAUTELA"
        else:
            outlook = "🟡 NEUTRO"

        print(f"{commodity:<15}: Pendiente {pendiente:7.4f} - Outlook: {outlook}")

    print("\n")

    # Commodities con mayor momentum
    momentum = {k: v for k, v in sorted(tendencias_recientes.items(),
                                      key=lambda item: item[1], reverse=True)}

    print("🚀 COMMODITIES CON MAYOR MOMENTUM ALCISTA:")
    top_3 = list(momentum.items())[:min(3, len(momentum))]
    for i, (commodity, pendiente) in enumerate(top_3):
        print(f"{i+1}. {commodity} (Pendiente: {pendiente:.4f})")

    print("\n")
    print("📉 COMMODITIES CON MAYOR PRESIÓN BAJISTA:")
    bottom_3 = list(momentum.items())[-min(3, len(momentum)):]
    for i, (commodity, pendiente) in enumerate(bottom_3):
        print(f"{i+1}. {commodity} (Pendiente: {pendiente:.4f})")

analisis_perspectivas(precios_cierre, performance)


# ## 💾 EXPORTACIÓN DE DATOS

# %%
# Exportación de resultados
def exportar_datos(precios, performance, volatilidad):
    """
    Exportación de datos para reportes externos
    """
    if precios is None or precios.empty:
        print("❌ No hay datos para exportar")
        return None

    # Crear DataFrame resumen
    resumen = pd.DataFrame({
        'Precio_Actual': precios.iloc[-1],
        'Precio_Inicial': precios.iloc[0],
        'Variacion_Porcentual': ((precios.iloc[-1] / precios.iloc[0] - 1) * 100)
    })

    # Agregar métricas de performance si están disponibles
    if performance is not None:
        resumen['Retorno_Total'] = performance['Retorno_Total (%)']
        resumen['Sharpe_Ratio'] = performance['Sharpe_Ratio']

    # Agregar volatilidad si está disponible
    if volatilidad is not None and not volatilidad.empty:
        resumen['Volatilidad_Actual'] = volatilidad.iloc[-1]

    # Exportar a CSV
    fecha_export = datetime.now().strftime("%Y%m%d_%H%M")
    nombre_archivo = f"informe_hidrocarburos_{fecha_export}.csv"
    resumen.to_csv(nombre_archivo)

    print(f"✅ Datos exportados exitosamente: {nombre_archivo}")
    print(f"📊 Resumen de métricas exportadas:")
    print(resumen.round(3))

    return resumen

resumen_final = exportar_datos(precios_cierre, performance, volatilidad)

# ## 🔍 ANÁLISIS ADICIONAL - DATOS GLP

# %%
# Función alternativa para obtener datos de GLP (Propano)
def obtener_datos_glp_alternativo():
    """
    Intentar obtener datos de GLP desde fuentes alternativas
    """
    print("\n🔍 BUSCANDO DATOS ALTERNATIVOS PARA GLP...")

    # Intentar con diferentes símbolos para GLP/Propano
    simbolos_glp = ['B0=F', 'PNG21.NYM', 'PA=F']  # Diferentes símbolos posibles

    for simbolo in simbolos_glp:
        try:
            glp_data = yf.download(simbolo, start=start_date, end=end_date, progress=False)
            if not glp_data.empty:
                print(f"✅ GLP encontrado con símbolo: {simbolo}")
                return glp_data, simbolo
        except:
            continue

    print("❌ No se pudieron obtener datos de GLP de fuentes estándar")
    return None, None

# Intentar obtener datos de GLP
glp_data, glp_symbol = obtener_datos_glp_alternativo()

if glp_data is not None:
    # Agregar GLP a los datos existentes
    datos_mercado['GLP_Propano'] = glp_data
    print(f"✅ GLP agregado al análisis con {len(glp_data)} registros")

    # Actualizar precios de cierre
    precio_col = 'Adj Close' if 'Adj Close' in glp_data.columns else 'Close'
    precios_cierre['GLP_Propano'] = glp_data[precio_col]

    # Recalcular análisis con GLP incluido
    print("\n🔄 ACTUALIZANDO ANÁLISIS CON DATOS DE GLP...")
    performance, retornos = analisis_performance(precios_cierre)
    volatilidad = calcular_volatilidad(precios_cierre)
    generar_informe_ejecutivo(precios_cierre, performance, volatilidad)


# # 🎯 CONCLUSIONES EJECUTIVAS
#
# ## **Puntos Clave del Informe:**
#
# 1. **📈 Tendencias Dominantes:** Identificación de commodities con momentum alcista/bajista
# 2. **🌪️ Niveles de Volatilidad:** Análisis de riesgo por commodity
# 3. **💰 Performance Relativa:** Comparativa de retornos entre diferentes hidrocarburos
# 4. **🔗 Correlaciones:** Relaciones entre precios de diferentes energéticos
# 5. **📊 Perspectivas:** Outlook basado en análisis técnico y tendencias recientes
#
# ## **Próximos Pasos Recomendados:**
#
# - Monitorear commodities con alta volatilidad para gestión de riesgo
# - Considerar diversificación basada en correlaciones identificadas
# - Revisar posiciones en commodities con tendencias bajistas persistentes
# - Actualizar estrategias de cobertura según niveles de correlación
#
# ---
# *Informe generado automáticamente - Gerencia de Mercados y Proyectos - Sector Energía*

 Librerías cargadas correctamente
📡 Conectando a fuentes de datos en tiempo real...
✅ Crudo_WTI: 126 registros - Columnas: [('Close', 'CL=F'), ('High', 'CL=F'), ('Low', 'CL=F'), ('Open', 'CL=F'), ('Volume', 'CL=F')]
✅ Crudo_Brent: 126 registros - Columnas: [('Close', 'BZ=F'), ('High', 'BZ=F'), ('Low', 'BZ=F'), ('Open', 'BZ=F'), ('Volume', 'BZ=F')]
✅ Gas_Natural: 126 registros - Columnas: [('Close', 'NG=F'), ('High', 'NG=F'), ('Low', 'NG=F'), ('Open', 'NG=F'), ('Volume', 'NG=F')]
✅ Gasolina: 126 registros - Columnas: [('Close', 'RB=F'), ('High', 'RB=F'), ('Low', 'RB=F'), ('Open', 'RB=F'), ('Volume', 'RB=F')]
✅ Fuel_Oil: 126 registros - Columnas: [('Close', 'HO=F'), ('High', 'HO=F'), ('Low', 'HO=F'), ('Open', 'HO=F'), ('Volume', 'HO=F')]
✅ Oro: 126 registros - Columnas: [('Close', 'GC=F'), ('High', 'GC=F'), ('Low', 'GC=F'), ('Open', 'GC=F'), ('Volume', 'GC=F')]
📊 ESTADÍSTICAS DESCRIPTIVAS - ÚLTIMOS 6 MESES
       Crudo_WTI  Crudo_Brent  Gas_Natural  Gasolina  Fuel_Oil      Oro
count     

🎯 MÉTRICAS DE PERFORMANCE - ÚLTIMOS 6 MESES
             Retorno_Total (%)  Volatilidad_Anual (%)  Sharpe_Ratio  \
Crudo_WTI               -0.016                 34.061         0.169   
Crudo_Brent              0.218                 31.111         0.169   
Gas_Natural             -8.728                 64.350         0.026   
Gasolina                -8.059                 28.095        -0.462   
Fuel_Oil                 7.971                 31.790         0.644   
Oro                     32.555                 20.341         2.897   

             Max_Drawdown (%)  
Crudo_WTI             -19.510  
Crudo_Brent           -16.751  
Gas_Natural           -32.414  
Gasolina              -20.541  
Fuel_Oil              -12.464  
Oro                    -6.742  


🎯 INFORME EJECUTIVO - MERCADO DE HIDROCARBUROS
📅 Período analizado: 2025-04-06 a 2025-10-03
⏰ Última actualización: 2025-10-03 21:47:55
📊 Commodities analizados: Crudo_WTI, Crudo_Brent, Gas_Natural, Gasolina, Fuel_Oil, Oro


💰 EVOLUCIÓN DE PRECIOS:
----------------------------------------
Crudo_WTI      :    60.69 USD (-0.02%) 🔴 BAJA
Crudo_Brent    :    64.35 USD (+0.22%) 🟢 ALZA
Gas_Natural    :     3.34 USD (-8.73%) 🔴 BAJA
Gasolina       :     1.86 USD (-8.06%) 🔴 BAJA
Fuel_Oil       :     2.23 USD (+7.97%) 🟢 ALZA
Oro            :  3912.10 USD (+32.56%) 🟢 ALZA


🏆 PERFORMANCE DESTACADA:
----------------------------------------
Mejor desempeño: Oro (32.56%)
Peor desempeño: Gas_Natural (-8.73%)


🌪️  VOLATILIDAD ACTUAL:
----------------------------------------
Crudo_WTI      :  24.38% (Media)
Crudo_Brent    :  24.04% (Media)
Gas_Natural    :  68.24% (Alta)
Gasolina       :  25.28% (Media)
Fuel_Oil       :  26.38% (Media)
Oro            :  13.45% (Baja)


💡 RECOMENDACIONES ESTRATÉGICAS:
-

🔮 PERSPECTIVAS Y OUTLOOK DE MERCADO


📈 TENDENCIAS RECIENTES (Últimos 30 días):
---------------------------------------------
Crudo_WTI      : Pendiente -0.0535 - Outlook: 🟡 NEUTRO
Crudo_Brent    : Pendiente -0.0440 - Outlook: 🟡 NEUTRO
Gas_Natural    : Pendiente  0.0131 - Outlook: 🟢 POSITIVO
Gasolina       : Pendiente -0.0075 - Outlook: 🟡 NEUTRO
Fuel_Oil       : Pendiente  0.0005 - Outlook: 🟢 POSITIVO
Oro            : Pendiente 16.5135 - Outlook: 🟢 POSITIVO


🚀 COMMODITIES CON MAYOR MOMENTUM ALCISTA:
1. Oro (Pendiente: 16.5135)
2. Gas_Natural (Pendiente: 0.0131)
3. Fuel_Oil (Pendiente: 0.0005)


📉 COMMODITIES CON MAYOR PRESIÓN BAJISTA:
1. Gasolina (Pendiente: -0.0075)
2. Crudo_Brent (Pendiente: -0.0440)
3. Crudo_WTI (Pendiente: -0.0535)
✅ Datos exportados exitosamente: informe_hidrocarburos_20251003_2147.csv
📊 Resumen de métricas exportadas:
             Precio_Actual  Precio_Inicial  Variacion_Porcentual  \
Crudo_WTI           60.690          60.700                -0.016   
Crudo_Bren

🎯 INFORME EJECUTIVO - MERCADO DE HIDROCARBUROS
📅 Período analizado: 2025-04-06 a 2025-10-03
⏰ Última actualización: 2025-10-03 21:47:55
📊 Commodities analizados: Crudo_WTI, Crudo_Brent, Gas_Natural, Gasolina, Fuel_Oil, Oro, GLP_Propano


💰 EVOLUCIÓN DE PRECIOS:
----------------------------------------
Crudo_WTI      :    60.69 USD (-0.02%) 🔴 BAJA
Crudo_Brent    :    64.35 USD (+0.22%) 🟢 ALZA
Gas_Natural    :     3.34 USD (-8.73%) 🔴 BAJA
Gasolina       :     1.86 USD (-8.06%) 🔴 BAJA
Fuel_Oil       :     2.23 USD (+7.97%) 🟢 ALZA
Oro            :  3912.10 USD (+32.56%) 🟢 ALZA
GLP_Propano    :      nan USD (+nan%) 🔴 BAJA


🏆 PERFORMANCE DESTACADA:
----------------------------------------
Mejor desempeño: Oro (32.56%)
Peor desempeño: Gas_Natural (-8.73%)


🌪️  VOLATILIDAD ACTUAL:
----------------------------------------
Crudo_WTI      :  24.38% (Media)
Crudo_Brent    :  24.04% (Media)
Gas_Natural    :  68.24% (Alta)
Gasolina       :  25.28% (Media)
Fuel_Oil       :  26.38% (Media)
Oro      